In [1]:
import tokenizers
import numpy as np
import torch

In [2]:
from tokenizers import BertWordPieceTokenizer

In [3]:
tokenizer = BertWordPieceTokenizer("bert-base-uncased-vocab.txt",
                                   lowercase=True,
                                   add_special_tokens=False,
                                   clean_text=True)

In [4]:
lines = []
textfile = '/home/tucker/data/imdb/aclImdb/train/unsup_all.txt'
#textfile = '/home/tucker/data/news/training-monolingual/news.2007.en.shuffled'
with open(textfile) as f:
  long_line = f.read().lower().replace('<br /><br />', ' ')

In [5]:
TOK_CHUNK_MAX = 58
TOK_CHUNK_MIN = 57
CHAR_CHUNK_MAX = 384
def GetExamplesFromSplitLine(line):
  token_ids = []
  tokens = line
  i = 0
  while i*TOK_CHUNK_MAX < len(tokens):
    potential = tokens[i*TOK_CHUNK_MAX: (i+1)*TOK_CHUNK_MAX]

    if len(potential)>=TOK_CHUNK_MIN:
      token_ids.append(" ".join(potential))
    i+=1

  return token_ids
  

In [7]:
half = len(long_line)//5

In [13]:
long_line[half+5]

' '

In [14]:
long_line = long_line[:half+5]

In [6]:
l=long_line.split()

In [7]:
del long_line

In [8]:
seq_chunks = []

t = GetExamplesFromSplitLine(l)
for seq in t:
  seq_chunks.append(seq)

In [18]:
torch.save(seq_chunks, "tmp.chunks")

In [4]:
seq_chunks = torch.load("tmp.chunks")

In [9]:
del l

In [10]:
seq_chunks[0]

'i admit, the great majority of films released before say 1933 are just not for me. of the dozen or so "major" silents i have viewed, one i loved (the crowd), and two were very good (the last command and city lights, that latter chaplin circa 1931). so i was apprehensive about this one, and humor is often'

In [11]:
len(seq_chunks)

200091

In [12]:
encoded = tokenizer.encode_batch(seq_chunks)

In [13]:
len(encoded)

200091

In [9]:
del seq_chunks

In [19]:
a=encoded[0]

In [14]:
token_len = 96
char_len = 384

In [15]:
idx_to_char_map = {v:k for k,v in torch.load('char_to_idx_map.pt').items()}

In [16]:
zerochar = idx_to_char_map[0]

In [17]:
def GetTokStartOffset(encoded):
  offsets = list([x for x in list(zip(*encoded.offsets))[0] if x<char_len])
  return offsets

In [18]:
token_list = []
char_list = []
offset_pos = []
for e in encoded:
  tokens = e.ids[:token_len]
  token_pad = token_len - len(tokens)
  tokens += [0]*token_pad

  chars = str(e.original_str)[:char_len]
  char_pad = char_len - len(chars)
  chars += zerochar*char_pad

  tok_start_idxs = GetTokStartOffset(e)

  offsets = torch.zeros((char_len,))
  offsets[tok_start_idxs]= 1

  token_list.append(tokens)
  char_list.append(chars)
  offset_pos.append(offsets)                        

In [19]:
char_list[501]

'and turkey, i have never had such a modern and timeless feeling from a silent movie. this results from the cool, timeless acting and characterizations, and the excellent directing and shooting. this is the first silent drammatic film my kids have thoroughly enjoyed, (until now only preferring silent comedic standards by keaton, chaplin, and harold lloyd.) i thought\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [20]:
data = {}
data['token_ids'] = torch.tensor(token_list).short()

In [21]:
data['chars'] = char_list
data['token_start_offsets'] = torch.stack(offset_pos).bool()

In [22]:
torch.save(data, 'imdb_tok96_char384.pt')

In [23]:
ds = torch.load('imdb_tok96_char384.pt')

In [24]:
del ds['token_ids']

In [25]:
torch.save(ds, 'imdb_tok96_char384_no_tok.pt')

In [41]:
ds = torch.load('news2007_tok96_char384_no_tok.pt')

In [26]:
char_to_idx_map  = torch.load('char_to_idx_map.pt')

In [27]:
encoded_chars = []

In [28]:
for seq in ds['chars']:
  enc = []
  for c in seq:
    enc.append(char_to_idx_map.get(c,0))
  encoded_chars.append(enc)

In [29]:
e = torch.tensor(encoded_chars)

In [30]:
ds['chars_encoded'] = e.char()

In [31]:
del ds['chars']

In [32]:
torch.save(ds, 'imdb_tok96_char384_enc_no_tok.pt')

In [33]:
ds

{'token_start_offsets': tensor([[ True, False,  True,  ..., False, False, False],
         [ True, False, False,  ..., False, False, False],
         [ True, False, False,  ..., False, False, False],
         ...,
         [ True, False, False,  ..., False, False, False],
         [ True, False, False,  ..., False, False, False],
         [ True, False, False,  ..., False, False, False]]),
 'chars_encoded': tensor([[48,  1, 40,  ...,  0,  0,  0],
         [43, 48, 45,  ...,  0,  0,  0],
         [45, 54, 57,  ...,  0,  0,  0],
         ...,
         [58, 48, 59,  ...,  0,  0,  0],
         [54, 53, 44,  ...,  0,  0,  0],
         [58, 54,  1,  ...,  0,  0,  0]], dtype=torch.int8)}

In [138]:
char_lens = np.array([len(e.original_str) for e in  encoded])

In [141]:
np.percentile(char_lens, 98)

382.0

In [142]:
char_lens.min()

181